In [ ]:
import requests
import time
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import pipeline
from prettytable import PrettyTable
import json
import nltk


In [2]:
# Download VADER lexicon
nltk.download('vader_lexicon')

# Initialize VADER and Hugging Face sentiment analysis
sia = SentimentIntensityAnalyzer()
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Fetch latest headlines
def fetch_latest_headlines(e867900d8528496fb5c2ba8fe1a6390b):
    url =f'https://newsapi.org/v2/top-headlines?country=us&apiKey={e867900d8528496fb5c2ba8fe1a6390b}'
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        return [article['title'] for article in articles]
    else:
        print(f"Error fetching news: {response.status_code}")
        return []



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\donel\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Analyze sentiment using TextBlob
def analyze_sentiment_textblob(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    sentiment = 'Positive' if polarity > 0 else 'Negative' if polarity < 0 else 'Neutral'
    return polarity, subjectivity, sentiment

# Analyze sentiment using VADER
def analyze_sentiment_vader(text):
    scores = sia.polarity_scores(text)
    compound_score = scores['compound']
    sentiment = 'Positive' if compound_score >= 0.05 else 'Negative' if compound_score <= -0.05 else 'Neutral'
    return compound_score, sentiment

# Analyze sentiment using Hugging Face Transformers
def analyze_sentiment_huggingface(text):
    result = sentiment_pipeline(text)[0]
    return result['label'], result['score']


In [ ]:
# Main function
def main():
    api_key = "e867900d8528496fb5c2ba8fe1a6390b"  # Replace with your NewsAPI.org key
    seen_headlines = set()

    while True:
        print("\nFetching latest news headlines...")
        headlines = fetch_latest_headlines(api_key)
        new_headlines = [headline for headline in headlines if headline not in seen_headlines]

        if not new_headlines:
            print("No new headlines. Waiting for the next fetch...")
            time.sleep(300)
            continue

        for headline in new_headlines:
            print(f"\nAnalyzing headline: {headline}")

            # Get sentiment analysis from all three methods
            tb_polarity, tb_subjectivity, tb_sentiment = analyze_sentiment_textblob(headline)
            vader_compound, vader_sentiment = analyze_sentiment_vader(headline)
            hf_label, hf_score = analyze_sentiment_huggingface(headline)

            # Create a table for the output
            table = PrettyTable()
            table.field_names = ["Method", "Sentiment", "Score/Polarity", "Confidence/Subjectivity"]

            table.add_row(["TextBlob", tb_sentiment, tb_polarity, tb_subjectivity])
            table.add_row(["VADER", vader_sentiment, vader_compound, "-"])
            table.add_row(["Hugging Face", hf_label, hf_score, "-"])

            print(table)

            # Add the headline to seen set
            seen_headlines.add(headline)

        print("\nWaiting for 5 minutes before the next fetch...")
        time.sleep(300)

if __name__ == "__main__":
    main()



Fetching latest news headlines...

Analyzing headline: Best Cyber Monday phone deals LIVE: iPhone 16, Samsung Galaxy S24, Pixel 9 biggest sales now - Tom's Guide
+--------------+-----------+--------------------+-------------------------+
|    Method    | Sentiment |   Score/Polarity   | Confidence/Subjectivity |
+--------------+-----------+--------------------+-------------------------+
|   TextBlob   |  Positive | 0.5681818181818181 |           0.4           |
|    VADER     |  Positive |       0.6369       |            -            |
| Hugging Face |  POSITIVE | 0.9963171482086182 |            -            |
+--------------+-----------+--------------------+-------------------------+

Analyzing headline: South Korean stocks rocked in U.S. trading after martial law declared - CNBC
+--------------+-----------+-------------------+-------------------------+
|    Method    | Sentiment |   Score/Polarity  | Confidence/Subjectivity |
+--------------+-----------+-------------------+---------